# PyTorch qGAN Implementation

Description

adapted from [PyTorch GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py)

In [21]:
# Necessary imports

import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F


from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.connectors import TorchConnector

# Set seed for random generators
algorithm_globals.random_seed = 42

- m samples - Gradient Lin Comb
- generic loss functions? for qGANs.
- Parameter from PyTorch
- PyTorch Discriminator, Loss
- Generator as samples to PyTorch Discriminator - Tensor? with custom gradient
- for grad qnn.backward(sampling=False, sparse=True)

In [22]:
import qiskit.tools.jupyter
%qiskit_version_table

In [23]:
# declare quantum instance
backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend)

### Load training data

In [24]:
# TODO: update
data_dim = 2

training_data = np.random.default_rng().multivariate_normal(mean=[0., 0.], cov=[[1, 0], [0, 1]], size=1000, check_valid='warn', 
                                                        tol=1e-8, method='svd')

batch_size = 100

dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True,)


### Definition of quantum generator

In [45]:
class Generator():
    def __init__(self):
#         super(Generator, self).__init__()
        
        qnn = QuantumCircuit(data_dim)
        qnn.h(qnn.qubits)
        ansatz = TwoLocal(data_dim, "ry", "cz", reps=1, entanglement="linear")
        qnn.compose(ansatz, inplace=True)
        self._parameters = nn.ParameterList([nn.Parameter(0) for i in range(len(ansatz.ordered_parameters))])
        #TODO: Interpret function to map output integer to range of data
        
        
        self._qnn = CircuitQNN(qnn, input_params=[], weight_params = ansatz.ordered_parameters, 
                               input_gradients=True, quantum_instance=qi, sampling=True, sparse=False, 
                               interpret=lambda x: f"{0:data_dim}".format(x)) 
        
        
#         self.qnn = TorchConnector(circuit_qnn)


    def forward(self):
        qnn_output = Tensor(self.qnn()._sample([], self.parameters))
        print('qnn_output ', qnn_output)
        #qnn_output.grad = 
        return qnn_output
    

### Definition of classical discriminator

In [46]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(data_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
            )

    def forward(self, input):
        
        return self.model(input)

### Definition of loss function and optimizer

In [47]:
# Loss function
g_loss_fun = nn.BCELoss()
#g_loss_fun.grad = 
d_loss_fun = nn.BCELoss()

#TODO overwrite PyTorch BCELoss gradient?

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

lr=0.0002
b1=0.5
b2=0.999
n_epochs=2

#TODO generator.parameters() replace with PyTorch parameter object

optimizer_G = Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

ValueError: Invalid format specifier

### Training

In [44]:
for epoch in range(n_epochs):
    for i, data in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(data.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(data.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_data = Variable(data.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_data = generator()

        # Loss measures generator's ability to fool the discriminator
        g_loss = loss(discriminator(gen_data), valid)
        # g_loss.backward(external_gradient=Tensor(...))
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_data), valid)
        fake_loss = adversarial_loss(discriminator(gen_data.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            #TODO: Do something like storing, printing
            pass

qnn_output  tensor([[10.],
        [10.],
        [10.],
        ...,
        [10.],
        [10.],
        [10.]], grad_fn=<_TorchNNFunctionBackward>)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1024x1 and 2x512)

### Open Questions

- Why do we want to support a sparse representation? The output of the quantum generator needs to have the same dimension as the input of the classical discriminator. If the dimensions are larger than what a classical representation can handle then we anyways have a problem.